# Timing Data

In [ ]:
from time_solvers import timed_trials, plot_solver_run_times
import numpy as np
import matplotlib.pyplot as plt
import gillespy2

In [ ]:
#Helper method for gathering imported items of a specific type:
def imported_classes(match_type):
    items = []
    key, value = None, None
    for key, value in globals().items():
        if isinstance(value, type) and issubclass(value, match_type) and value not in items:
            items.append(value)
    return items

In [ ]:
from gillespy2.example_models import *
#Trichloroethylene, LacOperon, Schlogl, MichaelisMenten, ToggleSwitch, Example, Tyson2StateOscillator
models = [MichaelisMenten(), Example(), Trichloroethylene()]#model() for model in imported_classes(gillespy2.Model)]
print('Imported models:')
for i, model in enumerate(models):
    print('{}:\t{}'.format(i+1, model.name))

In [ ]:
from gillespy2.solvers.python import *
#BasicSSASolver
from gillespy2.solvers.numpy import *
#BasicODESolver, BasicRootSolver, BasicTauLeapingSolver, NumPySSASolver, TauLeapingSolver
from gillespy2.solvers.cython import *
#CythonSSASolver
from gillespy2.solvers.cpp import *
#SSACSolver
from gillespy2.solvers.auto import *
#SSASolver
from gillespy2.solvers.stochkit import *
#StochKitODESolver, StochKitSolver
solvers = imported_classes(gillespy2.GillesPySolver)
solvers.remove(BasicODESolver)
solvers.remove(StochKitODESolver)
solvers.remove(TauLeapingSolver)
print('Imported solvers:')
for i, solver in enumerate(solvers):
    print('{}:\t{}'.format(i+1, solver.__name__))

In [ ]:
trajectories = [2**i for i in range(11)]
timing_data = timed_trials(models, solvers, trajectories, number_trials=30, output_file='timing_data.pkl')

In [ ]:
#Ideally, we can run the timed tests once and save the results for later in a .pkl file
import pickle
with open('timing_data.pkl', 'rb') as f:
    timing_data = pickle.load(f)
    


In [ ]:
%matplotlib inline
styles = {
    'StochKitSolver' : 'r*-',
    'SSACSolver' : 'g*-',
    'CythonSSASolver' : 'b*-',
    'NumPySSASolver' : 'k*-',
    'BasicTauHybridSolver' : 'y*-',
    'BasicTauLeapingSolver' : 'm*-'
}
plot_solver_run_times(timing_data, line_styles=styles)

In [ ]:
plot_solver_run_times(timing_data, line_styles=styles, ylabel='Log(seconds)', transformation=np.log)

In [ ]:
plot_solver_run_times(timing_data, line_styles=styles, ylabel='Relative Speed to Stochkit', baseline_solver_name=StochKitSolver.name)

In [ ]:
import matplotlib.pyplot as plt
ylabel='Average run time (s)'
reduce=np.mean
line_styles=styles

In [ ]:
model_names = timing_data.keys()
for model in model_names:
    plt.figure(figsize=(10,8))
    plt.title('{} Timing Results'.format(model))
    plt.xlabel('Trajectories', fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    baseline = None
    #if baseline_solver_name is not None:
    #    baseline = reduce(timing_data[model][baseline_solver_name][:, 1:], axis=1)
    #    if transformation is not None:
    #        baseline = transformation(times)
    solver_names = timing_data[model].keys()
    for solver in solver_names:
        trajectories = timing_data[model][solver][:,0]
        times = reduce(timing_data[model][solver][:, 1:], axis=1)
        #if transformation is not None:
        #    times = transformation(times)
        #if baseline is not None:
        #    times = np.divide(baseline, times)
        if solver in line_styles:
            plt.plot(trajectories, times, line_styles[solver], label=solver)
        else:
            plt.plot(trajectories, times, label=solver)
    plt.legend(loc='best', fontsize=16)


In [ ]:
ylabel='Average run time (s)'
reduce=np.mean
line_styles={
    'StochKitSolver' : 'r*-',
    'SSACSolver' : 'g*-',
    'CythonSSASolver' : 'b*-',
    'NumPySSASolver' : 'k*-',
    'BasicTauHybridSolver' : 'y*-',
    'BasicTauLeapingSolver' : 'm*-'
}
model_names = timing_data.keys()
for model in model_names:
    plt.figure(figsize=(10,8))
    plt.title('{} Timing Results'.format(model))
    plt.xlabel('Trajectories', fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    baseline = None
    #if baseline_solver_name is not None:
    #    baseline = reduce(timing_data[model][baseline_solver_name][:, 1:], axis=1)
    #    if transformation is not None:
    #        baseline = transformation(times)
    #solver_names = timing_data[model].keys()
    solver_names = ['StochKitSolver', 'NumPySSASolver', 'CythonSSASolver', 'SSACSolver']
    for solver in solver_names:
        trajectories = timing_data[model][solver][:,0]
        times = reduce(timing_data[model][solver][:, 1:], axis=1)
        #if transformation is not None:
        #    times = transformation(times)
        #if baseline is not None:
        #    times = np.divide(baseline, times)
        if solver in line_styles:
            plt.plot(trajectories, times, line_styles[solver], label=solver)
        else:
            plt.plot(trajectories, times, label=solver)
    plt.legend(loc='best', fontsize=16)
